In [1]:
import sys
sys.path.append('..')
from src.models.pipeline import pipeline
import glob
import json
import os
import pandas as pd
import ast
from datetime import datetime
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

In [2]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript
#print(transcripts_jsons)

'''Read the videos metadata to perform on them the segmentation'''
df_videos = pd.read_csv(topic_dataset_path)

''' Transfer topic shifts time to seconds units instead HH:MM:SS'''

def topic_shifts_seconds(topic_shifts):
    tp_shift_sec=[]
    for tp in topic_shifts:
        intervals = tp.split(':')
        seconds = int(intervals[2])
        minutes = int(intervals[1]) * 60
        hours = int(intervals[0]) * 60 *60
        tp_shift_sec.append(seconds + minutes + hours)
    return tp_shift_sec


for video_id in transcripts_jsons.keys():    
    df_videos.at[df_videos['video id'] == video_id,'topic shifts(ends)'] =\
    topic_shifts_seconds(\
                         df_videos[df_videos['video id'] == \
                                   video_id]['topic shifts(ends)'])

In [12]:
'''Create an array of functions to optimize'''
def build_function_to_optimize(workflow_label):
    pipeline_steps_values = workflow_label.split('-')
    steps_labels = ['segment','vectorize','similarity','added_filter','clustering']
    _pipeline = {}
    for key,value in list(zip(steps_labels,pipeline_steps_values)):
        _pipeline[key]=value
    
    '''takingcare of filters'''
    filter_params = {'filter_type': None,'mask_shape': None,'sim_thresh':None,'is_min_thresh':True}
    # if we have image filter 
    if _pipeline['added_filter'] != 'None':
        vals = _pipeline['added_filter'].split('_')
        filter_params['filter_type'] = vals[0]
        filter_params['mask_shape'] = ast.literal_eval(vals[1])
        
    #print(filter_params)    
    #print(_pipeline)
    '''Cosine and spectral clustering'''
    if _pipeline['segment'] =='sliding_window' and _pipeline['vectorize'] =='tfidf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        print("build function tfidf_cosine_sc")
        def _f(window_size,step_size,sim_thresh,n_clusters):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                window_size=window_size,step_size=step_size,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters},
                                accurrcy_shift=30
                               )

        return _f
    
    
    
    if _pipeline['segment'] =='sliding_window' and _pipeline['vectorize'] =='tf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        def _f(window_size,step_size,sim_thresh,n_clusters):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                window_size=window_size,step_size=step_size,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters}
                               )

        return _f
    
    if _pipeline['segment'] =='sliding_window' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'jensen_shannon' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(window_size,step_size,sim_thresh,
               n_clusters,alpha
               ,eta,chunksize,minimum_probability,passes
              ):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,
                                        slicing_method=_pipeline['segment'],
                                        window_size=window_size,step_size=step_size,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f
    
    if _pipeline['segment'] =='sliding_window' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(window_size,step_size,sim_thresh,
               n_clusters,alpha,
               eta,chunksize,minimum_probability,passes
              ):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,
                                        slicing_method=_pipeline['segment'],
                                        window_size=window_size,step_size=step_size,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f

    if _pipeline['segment'] =='sliding_window' and _pipeline['vectorize'] =='do_nothing' and _pipeline['similarity'] == 'wmdistance' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(window_size,step_size,sim_thresh,n_clusters):
            window_size = int(window_size)
            step_size = int(step_size)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            return pipeline.run_for_baye(groundbase,transcripts,
                                        slicing_method=_pipeline['segment'],
                                        window_size=window_size,step_size=step_size,
                                        vector_method=_pipeline['vectorize'],
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f 
    # ------------------------------------
    
    if _pipeline['segment'] =='audio' and _pipeline['vectorize'] =='tfidf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        print("build function tfidf_cosine_sc")
        def _f(silence_threshold,slice_length,step_size_audio,sim_thresh,n_clusters):
            silence_threshold = int(silence_threshold)
            slice_length = int(slice_length)
            step_size_audio = int(step_size_audio)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters},
                                accurrcy_shift=30
                               )

        return _f
    
    
    
    if _pipeline['segment'] =='audio' and _pipeline['vectorize'] =='tf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        def _f(silence_threshold,slice_length,step_size_audio,sim_thresh,n_clusters):
            silence_threshold = int(silence_threshold)
            slice_length = int(slice_length)
            step_size_audio = int(step_size_audio)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters}
                               )

        return _f
    
    if _pipeline['segment'] =='audio' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'jensen_shannon' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(silence_threshold,slice_length,step_size_audio,sim_thresh,
               n_clusters,alpha
               ,eta,chunksize,minimum_probability,passes
              ):
            silence_threshold = int(silence_threshold)
            slice_length = int(slice_length)
            step_size_audio = int(step_size_audio)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f
    
    if _pipeline['segment'] =='audio' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(silence_threshold,slice_length,step_size_audio,sim_thresh,
               n_clusters,alpha,
               eta,chunksize,minimum_probability,passes
              ):
            silence_threshold = int(silence_threshold)
            slice_length = int(slice_length)
            step_size_audio = int(step_size_audio)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f

    if _pipeline['segment'] =='audio' and _pipeline['vectorize'] =='do_nothing' and _pipeline['similarity'] == 'wmdistance' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(silence_threshold,slice_length,step_size_audio,sim_thresh,n_clusters):
            silence_threshold = int(silence_threshold)
            slice_length = int(slice_length)
            step_size_audio = int(step_size_audio)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f 
    
    #-----------------------
    
    if _pipeline['segment'] =='text_changing' and _pipeline['vectorize'] =='tfidf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        print("build function tfidf_cosine_sc")
        def _f(wanted_frequency,wanted_similarity_percent,sim_thresh,n_clusters):
            wanted_frequency = int(wanted_frequency)
            wanted_similarity_percent = int(wanted_similarity_percent)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters},
                                accurrcy_shift=30
                               )

        return _f
    
    
    
    if _pipeline['segment'] =='text_changing' and _pipeline['vectorize'] =='tf' and  _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        def _f(wanted_frequency,wanted_similarity_percent,sim_thresh,n_clusters):
            wanted_frequency = int(wanted_frequency)
            wanted_similarity_percent = int(wanted_similarity_percent)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh

            return pipeline.run_for_baye(groundbase,transcripts,
                                slicing_method=_pipeline['segment'],
                                silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                vector_method=_pipeline['vectorize'],
                                similarity_method=_pipeline['similarity'],
                                filter_params=filter_params,
                                clustering_params={'algorithm': _pipeline['clustering'],
                                                   'n_clusters':n_clusters}
                               )

        return _f
    
    if _pipeline['segment'] =='text_changing' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'jensen_shannon' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(wanted_frequency,wanted_similarity_percent,sim_thresh,
               n_clusters,alpha
               ,eta,chunksize,minimum_probability,passes
              ):
            wanted_frequency = int(wanted_frequency)
            wanted_similarity_percent = int(wanted_similarity_percent)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f
    
    if _pipeline['segment'] =='text_changing' and _pipeline['vectorize'] =='lda' and _pipeline['similarity'] == 'cosine' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(wanted_frequency,wanted_similarity_percent,sim_thresh,
               n_clusters,alpha,
               eta,chunksize,minimum_probability,passes
              ):
            wanted_frequency = int(wanted_frequency)
            wanted_similarity_percent = int(wanted_similarity_percent)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            passes = int(passes)
            chunksize = int(chunksize)
            minimum_probability = int(minimum_probability)
            vectorizing_params={"alpha":alpha,
                                'eta':eta,
                                'chunksize':chunksize,
                                'minimum_probability':minimum_probability,
                                'passes':passes,
                                'n_clusters':n_clusters}
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        vectorizing_params=vectorizing_params,
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f

    if _pipeline['segment'] =='text_changing' and _pipeline['vectorize'] =='do_nothing' and _pipeline['similarity'] == 'wmdistance' and _pipeline['clustering'] == 'spectral_clustering':
        #print("here you")
        def _f(wanted_frequency,wanted_similarity_percent,sim_thresh,n_clusters):
            wanted_frequency = int(wanted_frequency)
            wanted_similarity_percent = int(wanted_similarity_percent)
            n_clusters = int(n_clusters)
            filter_params['sim_thresh'] = sim_thresh
            filter_params['is_min_thresh'] = False
            return pipeline.run_for_baye(groundbase,transcripts,                                         
                                        slicing_method=_pipeline['segment'],
                                        silence_threshold=silence_threshold, slice_length=slice_length,step_size_audio=step_size_audio,
                                        vector_method=_pipeline['vectorize'],
                                        similarity_method=_pipeline['similarity'],
                                        filter_params=filter_params,
                                        clustering_params={'algorithm': _pipeline['clustering'],
                                                           'n_clusters':n_clusters}
                                       )
        return _f 

In [4]:
models_path  = '../models/bayesian_opt'
params_path = os.path.join(models_path,"parameters_bounds.json")
#print(params_path)
with open(params_path,'r') as f:
    param_bounds = ast.literal_eval(f.read())
#print(param_bounds)

In [13]:

''' Finding optimization for each video'''

workflows = list(param_bounds.keys())
workflows = list(filter(lambda x: 'tfidf' in x,workflows)) #[workflows[-1]]
print(workflows)
for workflow in workflows:
    print('Training workflow %s' %(workflow))
    function_to_optimized = build_function_to_optimize(workflow)
    '''Define the optimizer'''    
    optimizer = BayesianOptimization(
                f=function_to_optimized,
                pbounds=param_bounds[workflow],
                verbose=2,
                random_state=1
                )
    
    model_file_path = os.path.join(models_path,('%s.json' %(workflow)))
    if os.path.isfile(model_file_path):
        load_logs(optimizer,logs=[model_file_path])
    else:
        logger = JSONLogger(path=model_file_path)
        optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    
    '''optimizer.probe(
            params={'window_size':60,'step_size':20,'sim_thresh':0.4,
                    'alpha': 1e-05, 'eta': 0.005, 'chunksize': 100,
                    'minimum_probability': 0.0, 'passes': 6, 'n_clusters': 10},
            lazy=True,
    )'''

    for vid in transcripts_jsons.keys():
        '''Define the function to optimize'''
        video_metadata = df_videos.loc[df_videos['video id'] == vid]
        groundbase = video_metadata['topic shifts(ends)'].values.tolist()[:-1]
        transcripts = transcripts_jsons[vid]

        print("Running the algorithm on %s " %(vid))
        '''Run the algorithm'''
        optimizer.maximize(
            init_points = 2,
            n_iter = 5
        )
        
    print(optimizer.max)

['sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering', 'sliding_window-tfidf-cosine-median_(3,3)-spectral_clustering', 'sliding_window-tfidf-cosine-None-spectral_clustering']
Training workflow sliding_window-tfidf-cosine-median_(2,2)-spectral_clustering
build function tfidf_cosine_sc
Running the algorithm on 2mC1uqwEmWQ 
|   iter    |  target   | n_clus... | sim_th... | step_size | window... |
-------------------------------------------------------------------------
|  1        |  0.0      |  18.0     |  0.2      |  31.16    |  46.41    |
|  2        |  0.0      |  18.0     |  0.2      |  31.16    |  46.41    |
|  3        |  0.0      |  17.24    |  0.3782   |  59.72    |  197.9    |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  4        |  0.0      |  17.84    |  0.8651   |  35.14    |  107.5    |
|  5        |  0.4667   |  11.15    |  0.4821   |  10.13    |  45.7     |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  6        |  0.0      |  10.0     |  0.9      |  60.0     |  44.4     |


...

|  82       |  0.3846   |  12.87    |  0.4196   |  51.06    |  139.4    |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  83       |  0.0      |  12.0     |  0.6      |  60.0     |  116.7    |


C:\Users\yaniv\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


|  84       |  0.0      |  18.0     |  0.6      |  22.32    |  35.14    |
{'target': 0.8571428571428571, 'params': {'n_clusters': 18.0, 'sim_thresh': 0.6, 'step_size': 10.0, 'window_size': 47.55454708331205}}
